In [1]:
import sys  
sys.path.insert(0, '/home/yiyi/nlp_tm/models/DETM')

In [2]:
import torch
import os
import pickle
import csv
import numpy as np
import torch

In [ ]:
from __future__ import print_function

import argparse
import torch
import pickle 
import numpy as np 
import os 
import math 
import random 
import sys
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.io

import data 

from sklearn.decomposition import PCA
from torch import nn, optim
from torch.nn import functional as F

from detm import DETM
from utils import nearest_neighbors, get_topic_coherence

import pandas as pd

In [ ]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")


In [ ]:
root_dir ='/home/yiyi/nlp_tm/'

In [ ]:
num_topics = 10
filepath = os.path.join(root_dir, f'results/detm_twitter_K_10_Htheta_800_Optim_adam_Clip_0.0_ThetaAct_relu_Lr_0.005_Bsz_1000_RhoSize_300_L_3_minDF_10_trainEmbeddings_1_val_ppl_2757.5_epoch_3')

In [ ]:
model_path = os.path.join('/home/yiyi/nlp_tm/', filepath)

In [ ]:
with open(model_path, 'rb') as f:
    model = torch.load(f)

In [ ]:
model

In [ ]:
from utils import nearest_neighbors

In [ ]:
with open(os.path.join('/home/yiyi/nlp_tm/preprocessed_data', 'vocab.pkl'), 'rb') as f:
        vocab = pickle.load(f)

In [ ]:
len(vocab)

In [ ]:
def load_data(input_dir= os.path.join(root_dir, 'preprocessed_data')):
    token_file = os.path.join(input_dir, 'bow_tokens')
    count_file = os.path.join(input_dir, 'bow_counts')
    time_file = os.path.join(input_dir, 'bow_timestamps')
    tokens = scipy.io.loadmat(token_file)['tokens'].squeeze()
    counts = scipy.io.loadmat(count_file)['counts'].squeeze()
    times = scipy.io.loadmat(time_file)['timestamps'].squeeze()
    
    
    return tokens, counts, times

def _eta_helper(rnn_inp):
    inp = model.q_eta_map(rnn_inp).unsqueeze(1)
    hidden = model.init_hidden()
    output, _ = model.q_eta(inp, hidden)
    output = output.squeeze()
    etas = torch.zeros(model.num_times, model.num_topics).to(device)
    inp_0 = torch.cat([output[0], torch.zeros(model.num_topics,).to(device)], dim=0)
    etas[0] = model.mu_q_eta(inp_0)
    for t in range(1, model.num_times):
        inp_t = torch.cat([output[t], etas[t-1]], dim=0)
        etas[t] = model.mu_q_eta(inp_t)
    return etas

def get_theta(eta, bows):
    """
    Document proportions....
    """
    model.eval()
    
    with torch.no_grad():
        inp = torch.cat([bows, eta], dim=1)
        q_theta = model.q_theta(inp)
        mu_theta = model.mu_q_theta(q_theta)
        theta = F.softmax(mu_theta, dim=-1)
        return theta 


In [14]:
model.to(device)

DETM(
  (t_drop): Dropout(p=0.0, inplace=False)
  (theta_act): ReLU()
  (rho): Linear(in_features=300, out_features=20865, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=20875, out_features=800, bias=True)
    (1): ReLU()
    (2): Linear(in_features=800, out_features=800, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=800, out_features=10, bias=True)
  (logsigma_q_theta): Linear(in_features=800, out_features=10, bias=True)
  (q_eta_map): Linear(in_features=20865, out_features=200, bias=True)
  (q_eta): LSTM(200, 200, num_layers=3)
  (mu_q_eta): Linear(in_features=210, out_features=10, bias=True)
  (logsigma_q_eta): Linear(in_features=210, out_features=10, bias=True)
)

In [ ]:
tokens, counts, times = load_data()


In [29]:
tokens, counts, times = load_data()
print(len(tokens), len(counts), len(times))
print('documents nr: ', len(tokens))
indices = torch.split(torch.tensor(range(len(tokens))), 1000)

#     csvfile = open(f'topic_visualization_topnum{num_topics}.csv', 'w', newline='')
#     csvwriter = csv.writer(csvfile, delimiter= ',', quoting=csv.QUOTE_MINIMAL)
#     csvwriter.writerow(['Topic', 'Time', 'Topic Words'])

# theta_weights=[]


timeslist = [x for x in range(8)]
num_topics =10
num_words =10
# beta_arr = np.empty((len(tokens), num_topics, len(vocab)))
# gamma_dict = defaultdict(dict)
# for topic in range(0, 10):
#     gamma_dict[topic]={year:{} for year in range(0,8)}
#     for year in range(0,8):
#         gamma_dict[topic][year]= {x:0 for x in range(0, len(vocab))}
beta_rows = []
        
model.eval()
with torch.no_grad():
    alpha = model.mu_q_alpha
    rnn_inp = data.get_rnn_input(tokens, counts, times, 8, len(vocab), len(tokens))
    rnn_inp = rnn_inp.to(device)
    etas = _eta_helper(rnn_inp)


    for idx, ind in enumerate(indices):
        print(idx, len(ind))
        
        data_batch, times_batch = data.get_batch(
                        tokens, counts, ind, len(vocab),300, temporal=True, times=times)
        sums = data_batch.sum(1).unsqueeze(1)

        normalized_data_batch = data_batch / sums

        eta_td = etas[times_batch.type('torch.LongTensor')]
        print(eta_td)
        ############ get topic words#####
        alpha_td = alpha[:, times_batch.type('torch.LongTensor'), :].to(device)
        beta = model.get_beta(alpha_td).permute(1, 0, 2)
        print(beta.shape)
        for i in beta:
            beta_rows.append(i)
#         for k in range(num_topics):
#             for t in timeslist:
#                 gamma =beta[k,t,:]
#                 sorted_ids = g.argsort()[-num_words:][::-1]
#                 sorted_scores = np.sort(g)[-num_words:][::-1]
#                 d= zip(sorted_ids, sorted_scores)
#                 gamma_dict[k][t].update(d)

#                 del(gamma)
    
                
#         LEN = beta.shape[0]
#         print(idx*LEN, ':', (idx+1)*LEN)
#         beta_arr[idx*LEN:(idx+1)*LEN]= beta
                    
#         theta = get_theta(eta_td, normalized_data_batch).to(device)
#         theta = theta.cpu().numpy()
#         theta_weights.append(theta)


360097 360097 360097
documents nr:  360097
idx: 0/361
idx: 20/361
idx: 40/361
idx: 60/361
idx: 80/361
idx: 100/361
idx: 120/361
idx: 140/361
idx: 160/361
idx: 180/361
idx: 200/361
idx: 220/361
idx: 240/361
idx: 260/361
idx: 280/361
idx: 300/361
idx: 320/361
idx: 340/361
idx: 360/361
0 1000
tensor([[-0.7551,  0.0196, -0.0380,  ...,  1.4528, -0.8639,  0.3392],
        [-0.7551,  0.0196, -0.0380,  ...,  1.4528, -0.8639,  0.3392],
        [-0.7551,  0.0196, -0.0380,  ...,  1.4528, -0.8639,  0.3392],
        ...,
        [-0.7551,  0.0196, -0.0380,  ...,  1.4528, -0.8639,  0.3392],
        [-0.7551,  0.0196, -0.0380,  ...,  1.4528, -0.8639,  0.3392],
        [-0.7551,  0.0196, -0.0380,  ...,  1.4528, -0.8639,  0.3392]],
       device='cuda:3')
torch.Size([1000, 10, 20865])
1 1000
tensor([[-0.7551,  0.0196, -0.0380,  ...,  1.4528, -0.8639,  0.3392],
        [-0.7551,  0.0196, -0.0380,  ...,  1.4528, -0.8639,  0.3392],
        [-0.7551,  0.0196, -0.0380,  ...,  1.4528, -0.8639,  0.3392],
    

RuntimeError: CUDA out of memory. Tried to allocate 796.00 MiB (GPU 3; 11.91 GiB total capacity; 10.38 GiB already allocated; 207.06 MiB free; 11.09 GiB reserved in total by PyTorch)

In [19]:
beta_arr = np.array(beta_rows, dtype=np.uint8)

In [20]:
beta_arr.shape

(360097, 10, 20865)

In [26]:
beta_arr[:10]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [25]:
for k in range(num_topics):
    for t in timeslist:
        gamma = beta_arr[k, t, :]
        top_words = list(gamma.argsort()[-num_words+1:][::-1])
        topic_words = [vocab[a] for a in top_words]
#         topics_words.append(' '.join(topic_words))

#         csvwriter.writerow([k, t+2013, topic_words])

        print('Topic {} .. Time: {} ===> {}'.format(k, t, topic_words)) 


Topic 0 .. Time: 0 ===> ['indonesia', 'boozer', 'migraine', 'baltics', 'role', 'ammunition', 'europemustact', 'prater', 'taint']
Topic 0 .. Time: 1 ===> ['indonesia', 'boozer', 'migraine', 'baltics', 'role', 'ammunition', 'europemustact', 'prater', 'taint']
Topic 0 .. Time: 2 ===> ['indonesia', 'boozer', 'migraine', 'baltics', 'role', 'ammunition', 'europemustact', 'prater', 'taint']
Topic 0 .. Time: 3 ===> ['indonesia', 'boozer', 'migraine', 'baltics', 'role', 'ammunition', 'europemustact', 'prater', 'taint']
Topic 0 .. Time: 4 ===> ['indonesia', 'boozer', 'migraine', 'baltics', 'role', 'ammunition', 'europemustact', 'prater', 'taint']
Topic 0 .. Time: 5 ===> ['indonesia', 'boozer', 'migraine', 'baltics', 'role', 'ammunition', 'europemustact', 'prater', 'taint']
Topic 0 .. Time: 6 ===> ['indonesia', 'boozer', 'migraine', 'baltics', 'role', 'ammunition', 'europemustact', 'prater', 'taint']
Topic 0 .. Time: 7 ===> ['indonesia', 'boozer', 'migraine', 'baltics', 'role', 'ammunition', 'eur

In [ ]:
import csv
def visualize(num_topics, vocab):
    """Visualizes topics and embeddings and word usage evolution.
    """
    tokens, counts, times = load_data()
    print(len(tokens), len(counts), len(times))
    print('documents nr: ', len(tokens))
    indices = torch.split(torch.tensor(range(len(tokens))), 1000)
    
#     csvfile = open(f'topic_visualization_topnum{num_topics}.csv', 'w', newline='')
#     csvwriter = csv.writer(csvfile, delimiter= ',', quoting=csv.QUOTE_MINIMAL)
#     csvwriter.writerow(['Topic', 'Time', 'Topic Words'])
    
    theta_weights=[]
    beta_weights=[]
    
    model.eval()
    with torch.no_grad():
        alpha = model.mu_q_alpha
        rnn_inp = data.get_rnn_input(tokens, counts, times, 8, len(vocab), len(tokens))
        etas = _eta_helper(rnn_inp)
        
                
        for idx, ind in enumerate(indices):
            data_batch, times_batch = data.get_batch(
                            tokens, counts, ind, len(vocab),300, temporal=True, times=times)
            sums = data_batch.sum(1).unsqueeze(1)

            normalized_data_batch = data_batch / sums

            eta_td = etas[times_batch.type('torch.LongTensor')]
            ############ get topic words#####
            alpha_td = alpha[:, times_batch.type('torch.LongTensor'), :].to(device)
            beta = model.get_beta(alpha_td).permute(1, 0, 2)
            beta_weights.append(beta.cpu().numpy())
            
            theta = get_theta(eta_td, normalized_data_batch).to(device)
            theta = theta.cpu().numpy()
            theta_weights.append(theta)
    
    print('collected all beta weights...')
    beta_weights_list = []
    for w in beta_weights:
        for i in w:
            beta_weights_list.append(i)
    beta_weights_arr = np.stack(beta_weights_list, axis=0)
    print("beta shape: ", beta_weights_arr.shape)
    
    print('collected all theta weights...')
    theta_weights_list = []
    for w in theta_weights:
        for i in w:
            theta_weights_list.append(i)
    theta_weights_arr = np.stack(theta_weights_list, axis=0)
    print("theta shape: ", theta_weights_arr.shape)
    
    
    return beta_weights_arr, theta_weights_arr
#         topics_words = []
        
        
#         for k in range(num_topics):
#             for t in times:
#                 gamma = beta[k, t, :]
#                 top_words = list(gamma.cpu().numpy().argsort()[-num_words+1:][::-1])
#                 topic_words = [vocab[a] for a in top_words]
#                 topics_words.append(' '.join(topic_words))
                
#                 csvwriter.writerow([k, t+2013, topic_words])
                
#                 print('Topic {} .. Time: {} ===> {}'.format(k, t, topic_words)) 

#         print('\n')
#         print('Visualize word embeddings ...')
#         queries = ['immigrant', 'refugee']
#         try:
#             embeddings = model.rho.weight  # Vocab_size x E
#         except:
#             embeddings = model.rho         # Vocab_size x E
#         neighbors = []
#         for word in queries:
#             print('word: {} .. neighbors: {}'.format(
#                 word, nearest_neighbors(word, embeddings, vocab, num_words)))
#         print('#'*100)
#         return embeddings.cpu().numpy()

In [89]:
for k in range(num_topics):
    for t in timeslist:
        ga = gamma_dict[k][t]
        ga_sorted = {k:v for k,v in sorted(ga.items(), key=lambda item:item[1]) if v>0}
        topic_words = [vocab[a] for a in ga_sorted.keys()]
        
        print('Topic {} .. Time: {} ===> {}'.format(k, t+2013, topic_words)) 

Topic 0 .. Time: 2013 ===> ['unexpectedly', 'declaration', 'opec', 'eurovisiondebate', 'back', 'trumpmeltdown', 'shitshow', 'ungps', 'lobby', 'spanner']
Topic 0 .. Time: 2014 ===> ['unexpectedly', 'declaration', 'opec', 'eurovisiondebate', 'back', 'trumpmeltdown', 'shitshow', 'ungps', 'lobby', 'spanner']
Topic 0 .. Time: 2015 ===> ['unexpectedly', 'declaration', 'opec', 'eurovisiondebate', 'back', 'trumpmeltdown', 'shitshow', 'ungps', 'lobby', 'spanner']
Topic 0 .. Time: 2016 ===> ['unexpectedly', 'declaration', 'opec', 'eurovisiondebate', 'back', 'trumpmeltdown', 'shitshow', 'ungps', 'lobby', 'spanner']
Topic 0 .. Time: 2017 ===> ['unexpectedly', 'declaration', 'opec', 'eurovisiondebate', 'back', 'trumpmeltdown', 'shitshow', 'ungps', 'lobby', 'spanner']
Topic 0 .. Time: 2018 ===> ['unexpectedly', 'declaration', 'opec', 'eurovisiondebate', 'back', 'trumpmeltdown', 'shitshow', 'ungps', 'lobby', 'spanner']
Topic 0 .. Time: 2019 ===> ['unexpectedly', 'declaration', 'opec', 'eurovisiondeba

In [ ]:
gamma_weights[0]

In [ ]:
np.sort(gamma[0])[::-1][:10]

In [ ]:
from itertools import chain

In [ ]:
print('collected all beta weights...')
beta_weights_list = []
for w in beta_weights:
    for i in w:
        beta_weights_list.append(i)
with open('k_10/beta_weights_list.pkl', 'wb') as f:
    pickle.dump(beta_weights_list, f)


In [ ]:
beta_weights_arr = np.stack(beta_weights_list, axis=0)
print("beta shape: ", beta_weights_arr.shape)

print('collected all theta weights...')
theta_weights_list = []
for w in theta_weights:
    for i in w:
        theta_weights_list.append(i)
theta_weights_arr = np.stack(theta_weights_list, axis=0)
print("theta shape: ", theta_weights_arr.shape)

In [ ]:
len(beta_weights), beta_weights[0].shape

In [ ]:
len(theta_weights), 

In [ ]:
beta, theta = visualize(10, vocab)

In [ ]:
embeddings.shape

In [ ]:
keywords_ =['immigrant', 'refugee', 'immigration', 'asylum', 'exile', 'migrant', 'centroid']

In [ ]:
from sklearn import cluster
from sklearn import metrics

In [ ]:
# vectors of whole embeddings.
indexes_keywords = [vocab.index(word) for word in keywords_]
queries = [embeddings[index] for index in indexes_keywords] # vectors of the keywords

In [ ]:
kmeans = cluster.KMeans(n_clusters=1, random_state=0).fit(queries)

In [ ]:
centroid = kmeans.cluster_centers_[0]

In [ ]:
ranks= embeddings.dot(centroid).squeeze()
denom = centroid.T.dot(centroid).squeeze()
denom = denom*np.sum(embeddings*2, 1)
ranks = ranks/denom


In [ ]:
mostSimilar = []
[mostSimilar.append(idx) for idx in ranks.argsort()[::-1]]
nearest_neighbors = mostSimilar[:50]
nearest_neighbors = [vocab[comp] for comp in nearest_neighbors]

In [ ]:
nearest_neighbors

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
queries.append(centroid)

In [ ]:
len(queries)

In [ ]:
query_arr = np.array(queries)

In [ ]:
query_arr.shape

In [ ]:
len(keywords_)

In [ ]:
tsne = TSNE(n_components=2, random_state=0, n_iter=10000, perplexity=7)
T = tsne.fit_transform(query_arr)

In [ ]:
plt.figure(figsize=(16,10))
plt.scatter(T[:, 0], T[:, 1])
for label, x, y in zip(keywords_, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+2, y+2), xytext=(0, 0), textcoords='offset points')